# Team 2 
# Assignment 3 - Part I

## Modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pulp
from scipy import stats
sns.set()

## Load Data

In [2]:
# Demand (required bikes) of each Hub
dfDemands = pd.read_csv("demand_partI.csv", index_col=0)
dfDemands.reset_index(inplace=True)
dfDemands.head()

,Node,Demand
0,1,0
1,2,9
2,3,12
3,4,15
4,5,10


In [3]:
# Distances
dfDistances = pd.read_excel("distances_partI_new.xlsx", index_col=0)
dfDistances.reset_index(inplace=True)
dfDistances.head()

,i,j,Distances
0,1,1,0.000
1,1,2,3.223
2,1,3,2.743
3,1,4,2.206
4,1,5,0.310


## Sets

In [4]:
# 12 Hubs
hubs = list(dfDemands["Node"])
numhubs = len(hubs)

# Since we have 12 hubs, we have 12 time steps 
timesteps = list(range(1,numhubs+1))
numtimesteps = len(timesteps)

In [5]:
numhubs, numtimesteps

(12, 12)

## Parameters

In [6]:
# Parameters: Hub i to j distance (dictionary)
hubsdistance = {}
for i in hubs:
    for j in hubs:
        hubsdistance[(i,j)] = float(dfDistances[(dfDistances["i"] == i)&(dfDistances["j"] == j)]["Distances"])

In [7]:
# Parameters: Demand in hub j 
requiredbike = {}
for j in hubs:
    requiredbike[j] = float(dfDemands[(dfDemands["Node"]==j)]["Demand"])

## Variables

In [8]:
# Variables: if truck transferred form i to j in time t
xvar = pulp.LpVariable.dict("x", (hubs, hubs, timesteps), cat=pulp.LpBinary)

In [9]:
# Variables: number of bikes in the truck at time t
cvar = pulp.LpVariable.dict("c", (timesteps), lowBound=0.0,upBound=30.0, cat=pulp.LpInteger)

## Model Initialization

In [10]:
# Initialize model and objective sense
Model = pulp.LpProblem(name="Model", sense=pulp.LpMinimize)

## Objective Function

In [11]:
# Objective: Total distance
obj = pulp.lpSum([hubsdistance[(i,j)]*xvar[i,j,t] for i in hubs for j in hubs for t in timesteps])

In [12]:
# Add objective function to the model
Model += obj

## Constraints

In [13]:
# Contraint: Every hub just visited once
for j in hubs:
    Model += pulp.lpSum( [xvar[(i,j,t)] for t in timesteps for i in hubs] ) == 1

In [14]:
# Constraint: Conversation of flow
for j in hubs:
    for t in timesteps[:numtimesteps-1]:
        Model += pulp.lpSum( [xvar[(i,j,t)] for i in hubs] ) == pulp.lpSum( [xvar[(j,k,t+1)] for k in hubs] )

In [15]:
# Constraint: Truck Starts and ends at the Central Depot

# At t=1, the only outflow is from Central Depot
Model += pulp.lpSum( [xvar[(1,j,1)] for j in hubs] ) == 1 
for i in hubs[1:]:
    Model += pulp.lpSum( [xvar[(i,j,1)] for j in hubs] ) == 0

# At t=12, the only inflow is to Central Depot
Model += pulp.lpSum( [xvar[(i,1,numtimesteps)] for i in hubs] ) == 1 
for j in hubs[1:]:
    Model += pulp.lpSum( [xvar[(i,j,numtimesteps)] for i in hubs ]) == 0


In [16]:
# Constraint: Capacity of Truck is maximum 30 bikes

# At beginning of day, truck carries 0 bikes
Model += cvar[1] == 0

# Capacity of truck changes based on bike demand previous period
for t in timesteps[:numtimesteps-1]:
    Model += cvar[t+1] == cvar[t] + pulp.lpSum([requiredbike[j]*xvar[(i,j,t)] for i in hubs for j in hubs])

## Solution Process

In [17]:
# Write LP to file (optional, but often good to inspect model and find errors)
Model.writeLP("Model.lp")

# Solve model
Model.solve()
print("Status:", pulp.LpStatus[Model.status])

Status: Optimal


In [18]:
# Total cost
totalCost = pulp.value(Model.objective)
print("Total cost: " + str(totalCost))

Total cost: 23.333914


In [19]:
for t in timesteps:
    for i in hubs:
        for j in hubs:
            if xvar[(i,j,t)].varValue == 1.0:
                print("Timestep " + str(t) +": Hub " + str(i) +  " to Hub " + str(j) + 
                      " \n\t    truck capacity: " + str(cvar[t].varValue) +
                      " \n\t    distance travelled: " + str(hubsdistance[(i,j)]))                

Timestep 1: Hub 1 to Hub 5 
	    truck capacity: 0.0 
	    distance travelled: 0.31
Timestep 2: Hub 5 to Hub 11 
	    truck capacity: 10.0 
	    distance travelled: 4.38162
Timestep 3: Hub 11 to Hub 3 
	    truck capacity: 2.0 
	    distance travelled: 4.12802
Timestep 4: Hub 3 to Hub 6 
	    truck capacity: 14.0 
	    distance travelled: 0.779001
Timestep 5: Hub 6 to Hub 10 
	    truck capacity: 23.0 
	    distance travelled: 3.432
Timestep 6: Hub 10 to Hub 7 
	    truck capacity: 16.0 
	    distance travelled: 0.533349
Timestep 7: Hub 7 to Hub 4 
	    truck capacity: 3.0 
	    distance travelled: 0.736384
Timestep 8: Hub 4 to Hub 2 
	    truck capacity: 18.0 
	    distance travelled: 1.01701
Timestep 9: Hub 2 to Hub 9 
	    truck capacity: 27.0 
	    distance travelled: 2.50202
Timestep 10: Hub 9 to Hub 12 
	    truck capacity: 19.0 
	    distance travelled: 1.67627
Timestep 11: Hub 12 to Hub 8 
	    truck capacity: 12.0 
	    distance travelled: 1.8244
Timestep 12: Hub 8 to Hub 1 
	